# [Projet InPoDA - IN304](https://github.com/Egeyae/InPoDA-project-in304) - UVSQ UFR DES SCIENCES
#### *Done by KONSTANTINOV Julien and COSSEC Elouan*

**Goal:** Make a tweet analysis application *(extracting from french tweets: author, hastags, user mentioned, sentiment, topics)* and performing various data analysis actions


In [1]:
### Setup
from InPoDA_Pipeline import *

# The InPoDA_Pipeline class is used as an interface to use the project
# A logger is setup automatically, to remove any logging/log to a file, please update the config.json file
pipeline = InPoDAPipeline()
pipeline.logger.info("Pipeline setup was a success")

[2024-12-02 16:44:02,872] ::InPoDAPipeline:: (INFO) - Pipeline setup was a success


### I - Tweets data extraction

This part is a detailed explanation on how we extract tweets and parse them into a pandas.Dataframe()

##### ***1.** Load the tweets in memory*

In [2]:
tweets = pipeline.load_tweets()

[2024-12-02 16:44:02,875] ::InPoDAPipeline:: (INFO) - Loading tweets...


##### ***2.** Process the tweets in a pandas.DataFrame*

In [3]:
dataframe = pipeline.process_tweets_to_dataframe(tweets)

[2024-12-02 16:44:02,886] ::InPoDAPipeline:: (INFO) - Processing tweets into a DataFrame...


##### *(**3.** Annex: Sentiment Analysis)*

For learning purposes, we tried to create our own Neural Network model, trained to find the sentiment of a tweet. We used a Genetic Algorithm approach to explore solutions as we were not at ease with backpropagation. The training dataset is Sentiment140, around 1.6 millions tweets annoted for sentiment analysis. We embedded the training tweets using a multilingual model as the project tweets were in french and Sentiment140's are in english.

In the following cells, we try to present the global pipeline of model usage and training. However, training the model can cost a lot in terms of resources, so the code is commented by default.

###### **a.** Model loading

In [4]:
# Loads the best pre-computed model
pipeline.load_creature()

[2024-12-02 16:44:02,889] ::InPoDAPipeline:: (INFO) - Loading a pre-trained creature...


loaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaading


###### **b.** Model usage

In [5]:
# Example usage of the best pre-computed model
test_tweet = "I hate the current weather"

pipeline.process_input(test_tweet)

[2024-12-02 16:44:02,893] ::InPoDAPipeline:: (ERROR) - Best creature not loaded. Can't process input


###### (**c.** Model training)

In [6]:
# Loads as a pandas.DataFrame the training data
# Configuration can be updated in the `config.json` file

pipeline.load_training_data()
pipeline.data.head()

[2024-12-02 16:44:02,896] ::InPoDAPipeline:: (INFO) - Loading training data...
[2024-12-02 16:44:04,273] ::InPoDAPipeline:: (INFO) - Loaded training data with 1040 tweets.


,sentiment,id,date,query,user,text,embeddings
0,"[1.0, 0.0]",1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"http://twitpic.com/2y1zl - Awww, that's a bumm...","[-0.01171, -0.22, 0.1412, 0.001827, 0.259, 0.1..."
1,"[1.0, 0.0]",1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,"[0.1577, -0.10016, -0.32, 0.2515, 0.2886, 0.27..."
2,"[1.0, 0.0]",1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,I dived many times for the ball. Managed to sa...,"[-0.0796, -0.1976, -0.2961, 0.2598, 0.3318, -0..."
3,"[1.0, 0.0]",1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,"[0.04578, 0.006332, 0.24, 0.1765, 0.269, 0.060..."
4,"[1.0, 0.0]",1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"no, it's not behaving at all. i'm mad. why am ...","[-0.132, -0.2452, -0.2404, 0.1582, 0.08405, 0...."


In [ ]:
# Here we train a model based on the loaded data
# Since it is very expensive, it doesn't run by default
# ! Be aware that you need to update the save file in `config.json` if you don't want to overide pre-trained model !
run = False

if run:
    pipeline.train_genetic_algorithm()

[2024-12-02 16:44:04,287] ::InPoDAPipeline:: (INFO) - Training genetic algorithm...


### II - Tweets data analysis

This part is a presentation of different analysis we can extract from the data provided